In [1]:
%matplotlib inline

from __future__ import division
import numpy as np
import menpo.io as mio
from menpo.image import Image
from menpo.shape import PointUndirectedGraph
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

In [2]:
def coords_to_bb(coords):
    points = np.empty((4, 2))
    points[0] = coords[:2][::-1]
    points[1] = coords[[2, 1]][::-1]
    points[2] = coords[2:][::-1]
    points[3] = coords[[0, 3]][::-1]
    adjacency_matrix = np.array([[0, 1, 0, 1],
                                 [1, 0, 1, 0],
                                 [0, 1, 0, 1],
                                 [1, 0, 1, 0]])
    return PointUndirectedGraph(points, adjacency_matrix)

# Prepare Data

## Yale B

In [3]:
# import os
# import glob

# # Generate paths
# paths = [i for i in glob.glob('/Users/joan/Desktop/alaborticcv2015/yaleb/B*')]

# yaleb = []
# for p in paths:

#     # Load database:
#     images = []
#     for i in mio.import_images(p, verbose=True, max_images=None):
#         images.append(i)

#     # First image is the annotated template:
#     img = images[0]
#     labeller(img, 'PTS', ibug_face_66)

#     # Copy the annotations to the rest of the images:
#     for i in images:
#         i.landmarks = images[0].landmarks
#         lms = i.landmarks['PTS'].lms
#         bb = lms.bounding_box()
#         i.landmarks['bounding_box'] = bb
#         i.crop_to_landmarks_proportion_inplace(0.5, group='PTS')
#         i = i.rescale_landmarks_to_diagonal_range(200, group='PTS')

#     yaleb.append({'template': img, 'images': images})

In [4]:
from scipy.io.matlab import loadmat

dic = loadmat('/Users/joan/Desktop/alaborticcv2015/yaleb/myYaleCropped.mat')

templates = dic['tmplts'] / 255
images = dic['example_imgs'] / 255
coords = dic['coords'][0]

yaleb = []

step = 10
start = 0
for j in range(templates.shape[-1]):
    lms = coords_to_bb(coords)
    
    tmplt = Image(templates[..., j])
    tmplt.landmarks['bounding_box'] = lms
    tmplt.crop_to_landmarks_proportion_inplace(0.5)
    tmplt = tmplt.rescale_landmarks_to_diagonal_range(200)
    
    finish = start + step
    imgs = []
    for k in range(start, finish):
        img = Image(images[..., k])
        img.landmarks['bounding_box'] = lms
        img.crop_to_landmarks_proportion_inplace(0.5)
        img = img.rescale_landmarks_to_diagonal_range(200)
        imgs.append(img)
    
    start = finish
    
    yaleb.append({'template': tmplt, 'images': imgs})

In [5]:
mio.export_pickle(yaleb, '/Users/joan/Desktop/alaborticcv2015/data/yaleb.pkl.gz', overwrite=True)

## AR 

In [10]:
from scipy.io.matlab import loadmat

dic = loadmat('/Users/joan/Desktop/alaborticcv2015/ar/myARCropped.mat')

templates = dic['tmplts'][0]
images = dic['example_imgs'][0]
coords = dic['coords']

ar = []

step = 3
start = 0
for t in templates:
    lms = coords_to_bb(coords[start])
    
    tmplt = Image(t / 255)
    tmplt.landmarks['bounding_box'] = lms
    tmplt.crop_to_landmarks_proportion_inplace(0.5)
    tmplt = tmplt.rescale_landmarks_to_diagonal_range(200)
    
    finish = start + step
    imgs = []
    for i in images[start:finish]:
        img = Image(i / 255)
        img.landmarks['bounding_box'] = lms
        img.crop_to_landmarks_proportion_inplace(0.5)
        img = img.rescale_landmarks_to_diagonal_range(200)
        imgs.append(img)
    
    start = finish
    
    ar.append({'template': tmplt, 'images': imgs})

In [11]:
mio.export_pickle(ar, '/Users/joan/Desktop/alaborticcv2015/data/ar.pkl.gz', overwrite=True)

## Classic 

In [8]:
classic = []

# Load database:
images = []
for i in mio.import_images('/Users/joan/Desktop/alaborticcv2015/other/', 
                           verbose=True, max_images=None):
    images.append(i)
    
img = images[0]
img.landmarks['bounding_box'] = coords_to_bb(np.array([280, 240, 400, 380]))
classic.append({'car': img})

img = images[1]
img.landmarks['bounding_box'] = coords_to_bb(np.array([180, 180, 350, 350]))
classic.append({'knee': img})

img = images[3]
img.landmarks['bounding_box'] = coords_to_bb(np.array([70, 20, 175, 130]))
classic.append({'takeo': img})

- Loading 4 assets: [====================] 100%

In [9]:
mio.export_pickle(classic, '/Users/joan/Desktop/alaborticcv2015/data/classic.pkl.gz', overwrite=True)